## Install neccessary packages

In [ ]:
# Python 3.11.13
!python --version

In [ ]:
# %pip install -r ./extract_text/requirements.txt
# %pip install -r ./text_normalizer_vietnamese/requirements.txt
# %pip install pymupdf
# %pip install matplotlib pandas
# %pip install -r ./sentence_alignment/requirements.txt

In [ ]:
%pip install -r requirements.txt
%pip install -r ./quocngu_normalizer/requirements.txt

In [ ]:
%pip install matplotlib

In [ ]:
%pip install sentence_transformers

In [ ]:
%pip install -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import re
from pathlib import Path
from typing import Literal

from pdf_extractor import SinoNomPDFExtractor, QuocNguPDFExtractor
from quocngu_preprocessing import quocngu_preprocessing
from sinonom_preprocessing import sinonom_preprocessing

from utils import extract_quocngu_section_number, extract_sino_section_number, convert_list_to_dict
from alignment import align_section

In [ ]:
PAGE_BREAK = "\f\n"
PARAGRAPH_BREAK = "\n\n"
SENTENCE_BREAK = "\n"
CHINESE_CHAPTER_TEMPLATE = r"^第[一二三四五六七八九十百千零〇○]+[回囘]"
VIETNAMESE_CHAPTER_TEMPLATE = r"^HỒI THỨ(?: [\wÀ-Ỵ]+)+$"

QN_input_file = "./data/source/Book-Tay_Du_Ky-Viet.pdf"
SN_input_file = "./data/source/Book-Tay_Du_Ky-Trung.pdf"

QN_raw_folder = "./data/raw/QuocNgu"
SN_raw_folder = "./data/raw/SinoNom"

QN_preprocessed_folder = "./data/preprocessed/QuocNgu"
SN_preprocessed_folder = "./data/preprocessed/SinoNom"

xml_output_folder = "./data/output/"

In [ ]:
%load_ext autoreload
%autoreload 2

sino_extractor = SinoNomPDFExtractor(SN_input_file, 1, 10)
quocngu_extractor = QuocNguPDFExtractor(QN_input_file, 48, 50)

# =============================================
# Get raw text
sino_sections = sino_extractor.get_splitted_sections(CHINESE_CHAPTER_TEMPLATE)
quocngu_sections = quocngu_extractor.get_splitted_sections(VIETNAMESE_CHAPTER_TEMPLATE)

# save_quocngu_sections(Path(QN_raw_folder), quocngu_sections, True)

# print(quocngu_sections[0])

# =============================================
# Preprocessing
sinonom_sentence_dict = {}
for section in sino_sections:
    section_num = extract_sino_section_number(section)
    section = sinonom_preprocessing(section)
    sinonom_sentence_dict[section_num] = convert_list_to_dict(section)
# print(sinonom_sentence_dict)

quocngu_sentence_dict = {}
for section in quocngu_sections:
    section_num = extract_quocngu_section_number(section)
    section = quocngu_preprocessing(section)
    quocngu_sentence_dict[section_num] = section
# print(quocngu_sentence_dict)

# =============================================
# Sentence Alignment
for key in sinonom_sentence_dict:
    if key in quocngu_sentence_dict.keys():
        results = align_section(sinonom_sentence_dict[key], quocngu_sentence_dict[key])
        break

# =============================================
# Save output

## 1. Lấy dữ liệu văn bản từ tệp PDF

In [ ]:
# VĂN BẢN CHỮ QUỐC NGỮ
# !python ./pdf_to_text/main.py -p ./data/source/Book-Tay_Du_Ky-Viet.pdf -l viet -o ./data/raw_text/vietnamese -fp 48 -np 2 --rule_path ./pdf_to_text/removal_rules_vietnamese.json --use_rule
# Trang chứa nội dung: 48 -> 3415

In [ ]:
# VĂN BẢN CHỮ HÁN
# !python ./pdf_to_text/main.py -p ./data/source/Book-Tay_Du_Ky-Trung.pdf -l trung -o ./data/raw_text/chinese -fp 1 -np 2 --rule_path ./pdf_to_text/removal_rules_chinese.json --use_rule --use_page_split
# !python ./extract_text/extract_text_from_pdf.py
# Trang chứa nội dung: 1 -> 542

## 2. Chuẩn hóa dữ liệu văn bản 

### 2.1 Chuẩn hóa văn bản chữ Quốc ngữ

In [ ]:
# !python ./text_normalizer_vietnamese/main.py ./data/raw_text/vietnamese/ ./data/cleaned_text/vietnamese

### 2.2 Chuẩn hóa văn bản chữ Hán

In [ ]:
# !python ./text_normalizer_chinese/main.py -i ./data/raw_text/chinese/ -o ./data/cleaned_text/chinese

In [ ]:
# !python ./sentence_alignment/sentence_alignment.py

In [ ]:
# %pip install pdfplumber

In [ ]:
# %pip install "numpy<=2.0.0" --force-reinstall

In [1]:
# %load_ext autoreload
# %autoreload 2

import os
import re
from pathlib import Path
from typing import Literal

from pdf_extractor import SinoNomPDFExtractor, QuocNguPDFExtractor
from quocngu_preprocessing import quocngu_preprocessing
from sinonom_preprocessing import sinonom_preprocessing, split_chinese_sentences, normalize_chinese_text
from utils import extract_quocngu_section_number, extract_sino_section_number, convert_list_to_dict
from alignment import align_section

d:\Application\Miniconda\envs\nlp_GK\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PAGE_BREAK = "\f\n"
PARAGRAPH_BREAK = "\n\n"
SENTENCE_BREAK = "\n"
CHINESE_CHAPTER_TEMPLATE = r"^第[一二三四五六七八九十百千零〇○]+[回囘]"
VIETNAMESE_CHAPTER_TEMPLATE = r"^HỒI THỨ(?: [\wÀ-Ỵ]+)+$"

QN_input_file = "./data/source/Book-Tay_Du_Ky-Viet.pdf"
SN_input_file = "./data/source/Book-Tay_Du_Ky-Trung.pdf"

QN_raw_folder = "./data/raw/QuocNgu"
SN_raw_folder = "./data/raw/SinoNom"

QN_preprocessed_folder = "./data/preprocessed/QuocNgu"
SN_preprocessed_folder = "./data/preprocessed/SinoNom"

xml_output_folder = "./data/output/"

In [ ]:
sino_extractor = SinoNomPDFExtractor(SN_input_file, 1, 10)
quocngu_extractor = QuocNguPDFExtractor(QN_input_file, 48, 50)

# =============================================
# Get raw text
sino_sections = sino_extractor.get_splitted_sections(CHINESE_CHAPTER_TEMPLATE)
quocngu_sections = quocngu_extractor.get_splitted_sections(VIETNAMESE_CHAPTER_TEMPLATE)

# save_quocngu_sections(Path(QN_raw_folder), quocngu_sections, True)

# =============================================
# Preprocessing
sinonom_sentence_dict = {}
for section in sino_sections:
    section_num = extract_sino_section_number(section)
    section = sinonom_preprocessing(section)
    sinonom_sentence_dict[section_num] = convert_list_to_dict(section)
print(sinonom_sentence_dict)

quocngu_sentence_dict = {}
for section in quocngu_sections:
    section_num = extract_quocngu_section_number(section)
    section = quocngu_preprocessing(section)
    quocngu_sentence_dict[section_num] = section
print(quocngu_sentence_dict)

# =============================================
# Sentence Alignment

import json

for key in sinonom_sentence_dict:
    if key in quocngu_sentence_dict.keys():
        aligned_section = align_section(sinonom_sentence_dict[key], quocngu_sentence_dict[key], int(key))
        with open("output.json", 'w') as fp:
            json.dump(aligned_section, fp)
        break
    
# =============================================
# Save output


{1: {'000.01': '第一回靈根育孕源流出心性修持大道生', '001.01': '詩曰:混沌未分天地亂，茫茫渺渺無人見。', '001.02': '自從盤古破鴻濛，開闢從茲清濁辨。', '001.03': '覆載群生仰至仁，發明萬物皆成善。', '001.04': '欲知造化會元功，須看西遊釋厄傳。', '002.01': '蓋聞天地之數，有十二萬九千六百歲為一元。', '002.02': '將一元分為十二會，乃子，丑，寅，卯，辰，巳，午，未，申，酉', '002.03': '戌，亥之十二支也。', '002.04': '每會該一萬八百歲。', '002.05': '且就一日而論，子時得陽氣，而丑則雞鳴寅不通光', '002.06': '而卯則日出辰時食後，而巳則挨排日午天中，而未則西蹉申時晡', '002.07': '而日落酉，戌黃昏，而人定亥。', '002.08': '譬於大數，若到戌會之終，則天地昏曚而萬物否矣。', '002.09': '再去五千四百歲，交亥會之初，則當黑暗，而兩間人物俱無矣', '002.10': '故曰混沌。', '002.11': '又五千四百歲，亥會將終，貞下起元，近子之會，而復逐漸開明。', '002.12': '邵康節曰:“冬至子之半，天心無改移。', '002.13': '一陽初動處，萬物未生時。”', '002.14': '到此，天始有根。', '002.15': '再五千四百歲，正當子會，輕清上騰，有日，有月，有星，有辰。', '002.16': '日、月、星、辰，謂之四象。', '002.17': '故曰，天開於子。', '002.18': '又經五千四百歲，子會將終，近丑之會，而逐漸堅實。', '002.19': '《易》曰:“大哉乾元至哉坤元萬物資生，乃順承天。”', '002.20': '至此，地始凝結。', '002.21': '再五千四百歲，正當丑會，重濁下凝，有水，有火，有山，有石', '002.22': '有土。', '002.23': '水、火、山、石、土，謂之五形。', '002.24': '故曰，地闢於丑。', '002.25': '又經五千四百歲，丑會終而寅會之初，發生萬物。', '002.26': '曆曰:“天氣下降，地氣上升天地交合，群物皆生。”', '002.27': '至此，天清